In [1]:
import sympy
import math
from sympy import init_printing, Matrix, MatMul, integrate, symbols, eye

init_printing(use_latex='mathjax')

v_s, v_p, v_a = symbols('sigma_s^2 sigma_p^2 sigma_a^2')
dt, phi = symbols('\Delta{t} \Phi_s')

## State Vector and Evolution

Use $ ( x, y, v_f, v_s, \theta, \omega )$

## Single linear (position, velocity)

See rlabbe on Github

In [2]:
v = Matrix([[dt**2 / 2], [dt]])
Q_s = v @ v_s @ v.T
print('Q_s =')
Q_s

Q_s =


⎡         4               3    ⎤
⎢\Delta{t} ⋅σ²ₛ  \Delta{t} ⋅σ²ₛ⎥
⎢──────────────  ──────────────⎥
⎢      4               2       ⎥
⎢                              ⎥
⎢         3                    ⎥
⎢\Delta{t} ⋅σ²ₛ           2    ⎥
⎢──────────────  \Delta{t} ⋅σ²ₛ⎥
⎣      2                       ⎦

Each pos,velocity pair as a similar 2x2 matrix. Note, use different uncertainties for parallel ("s") and perpendicular velocities, since strafing is highly restricted in a normal tank robot.

Here is the full 6-dimensional Q matrix.

In [3]:
Q_spa = eye(6)
Q_spa[0:2, 0:2] = v @ v_s @ v.T
Q_spa[2:4, 2:4] = v @ v_p @ v.T
Q_spa[4:6, 4:6] = v @ v_a @ v.T

Q_spa

⎡         4               3                                                   
⎢\Delta{t} ⋅σ²ₛ  \Delta{t} ⋅σ²ₛ                                               
⎢──────────────  ──────────────        0               0               0      
⎢      4               2                                                      
⎢                                                                             
⎢         3                                                                   
⎢\Delta{t} ⋅σ²ₛ           2                                                   
⎢──────────────  \Delta{t} ⋅σ²ₛ        0               0               0      
⎢      2                                                                      
⎢                                                                             
⎢                                         4               3                   
⎢                                \Delta{t} ⋅σ²ₚ  \Delta{t} ⋅σ²ₚ               
⎢      0               0         ──────────────  ───

## Swap the columns

I want to use the column order of (x, y, v_s, v_p, angle, v_angle), so swap the columns. (Does not really change the math!)

In [4]:
T = eye(6)
T[1, 1] = 0
T[1, 2] = 1
T[2, 2] = 0
T[2, 1] = 1
T

⎡1  0  0  0  0  0⎤
⎢                ⎥
⎢0  0  1  0  0  0⎥
⎢                ⎥
⎢0  1  0  0  0  0⎥
⎢                ⎥
⎢0  0  0  1  0  0⎥
⎢                ⎥
⎢0  0  0  0  1  0⎥
⎢                ⎥
⎣0  0  0  0  0  1⎦

In [5]:
Q_spa_2 = T @ Q_spa @ T.T
Q_spa_2

⎡         4                               3                                   
⎢\Delta{t} ⋅σ²ₛ                  \Delta{t} ⋅σ²ₛ                               
⎢──────────────        0         ──────────────        0               0      
⎢      4                               2                                      
⎢                                                                             
⎢                         4                               3                   
⎢                \Delta{t} ⋅σ²ₚ                  \Delta{t} ⋅σ²ₚ               
⎢      0         ──────────────        0         ──────────────        0      
⎢                      4                               2                      
⎢                                                                             
⎢         3                                                                   
⎢\Delta{t} ⋅σ²ₛ                           2                                   
⎢──────────────        0         \Delta{t} ⋅σ²ₛ     

# Rotate to (x,y)

Now, rotate the "basis" so that the first two position columns are (x, y), instead of (forward, perpendicular). Leave the other columns alone.

$\theta$ is the yaw angle; that is, the angle between the robot forward direction and the x-axis of the field

In [6]:
a = symbols('theta')
T = eye(6)
T[0, 0] = T[1, 1] = sympy.cos(a)
T[1, 0] = sympy.sin(a)
T[0, 1] = -T[1, 0]
T

⎡cos(θ)  -sin(θ)  0  0  0  0⎤
⎢                           ⎥
⎢sin(θ)  cos(θ)   0  0  0  0⎥
⎢                           ⎥
⎢  0        0     1  0  0  0⎥
⎢                           ⎥
⎢  0        0     0  1  0  0⎥
⎢                           ⎥
⎢  0        0     0  0  1  0⎥
⎢                           ⎥
⎣  0        0     0  0  0  1⎦

In [7]:
Q_xya = T @ Q_spa_2 @ T.T
Q_xya

⎡                4        2               4        2                       4  
⎢       \Delta{t} ⋅σ²ₚ⋅sin (θ)   \Delta{t} ⋅σ²ₛ⋅cos (θ)           \Delta{t} ⋅σ
⎢       ────────────────────── + ──────────────────────         - ────────────
⎢                 4                        4                                  
⎢                                                                             
⎢           4                              4                                  
⎢  \Delta{t} ⋅σ²ₚ⋅sin(θ)⋅cos(θ)   \Delta{t} ⋅σ²ₛ⋅sin(θ)⋅cos(θ)         \Delta{
⎢- ──────────────────────────── + ────────────────────────────         ───────
⎢               4                              4                              
⎢                                                                             
⎢                             3                                               
⎢                    \Delta{t} ⋅σ²ₛ⋅cos(θ)                                    
⎢                    ─────────────────────          